In [27]:
import cv2 as cv
from imutils import perspective
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sudoku_solver as ss
model = tf.keras.models.load_model('digitRecognizer-v2.model')

In [51]:
#Load image and apply gaussian blur
img = cv.imread('sudoku.jfif')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
gblur = cv.GaussianBlur(gray, (9,9), 0)
cv.imshow('gblur', gblur)
cv.waitKey(0)
cv.destroyAllWindows()

In [29]:
#Apply edge detection, remove lighter lines and dilate edges
thresh = cv.adaptiveThreshold(gblur, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 11, 2)
cv.imshow('thresh', thresh)
cv.waitKey(0)
cv.destroyAllWindows()

In [30]:
#Find contours on the image
contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

In [36]:
#Find the 4-sided contour with largest area and map it to a square of 600x600
max_area = 0
max_rect = 0
for contour in contours:
    approx = cv.approxPolyDP(contour, 0.01*cv.arcLength(contour, True), True)
    if len(approx) == 4:
        area = cv.contourArea(approx)
        if area > max_area:
            max_area = area
            max_rect = approx
pts1 = np.squeeze(max_rect).tolist()
pts1 = np.float32(pts1)
cv.circle(img, tuple(pts1[0]), 5, (0,255,0), -1)
cv.circle(img, tuple(pts1[1]), 5, (0,255,0), -1)
cv.circle(img, tuple(pts1[2]), 5, (0,255,0), -1)
cv.circle(img, tuple(pts1[3]), 5, (0,255,0), -1)
pts1 = perspective.order_points(pts1)
pts2 = np.float32([[0,0], [450, 0], [450,450], [0,450]])
matrix = cv.getPerspectiveTransform(pts1, pts2)
result = cv.warpPerspective(gray, matrix, (450, 450))
color_result = cv.warpPerspective(img, matrix, (450, 450))

cv.imshow('result', result)
cv.imshow('contours', gray)
cv.imshow('image', img)
cv.waitKey(0)
cv.destroyAllWindows()

In [37]:
# Get top-left coordinates of each square
squares = []
for i in range(9):
    for j in range(9):
        p1 = (j*50, i*50)
        squares.append(p1)

In [38]:
# Crop the square
thresh = cv.adaptiveThreshold(result, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 11, 5)
roi = []
color_roi = []
for x, y in squares:
    roi.append(cv.morphologyEx(thresh[y+5:y+45, x+5:x+45], cv.MORPH_OPEN, np.ones((2,2), np.uint8)))
    color_roi.append(color_result[y+5:y+45, x+5:x+45])
%matplotlib qt
for i in range(81):
    plt.subplot(9, 9, i+1), plt.imshow(color_roi[i], cmap='gray')
    plt.title(i)
    plt.axis('off')

In [39]:
# If the number of white pixels is greater than threshold value then they contain digits
threshold = 50
digits = []
idx = []
for i in range(81):
    num_white_pixels = np.sum(roi[i][10:30, 10:30] == 255)
    if num_white_pixels > threshold:
        digits.append(cv.bitwise_not(roi[i]).reshape(40,40,1))
        idx.append(i)
for digit in digits:
   cv.imshow('', digit)
   cv.waitKey(0)
   cv.destroyAllWindows()

In [41]:
predictions = []
for digit in digits:
    predictions.append(int(model.predict_classes(digit.reshape(-1,40,40,1)/255.)))
plt.figure()
for i in range(len(digits)):
    plt.subplot(6, 6, i+1), plt.imshow(digits[i].reshape(40,40), cmap='gray')
    plt.title(predictions[i])
    plt.axis('off')
plt.plot()

[]

In [42]:
grid = ss.build_grid(predictions, idx)
if ss.solve(grid):
    print(grid)
else:
    print('error')

[[4 6 8 1 7 2 9 3 5]
 [1 9 3 6 8 5 4 7 2]
 [7 5 2 9 4 3 8 6 1]
 [3 7 6 5 9 4 2 1 8]
 [2 1 5 8 3 7 6 9 4]
 [8 4 9 2 6 1 3 5 7]
 [6 2 7 4 5 9 1 8 3]
 [9 3 1 7 2 8 5 4 6]
 [5 8 4 3 1 6 7 2 9]]


In [43]:
for i in range(81):
    if i in idx: continue
    row = int(i/9)
    col = i%9
    x, y = squares[i]
    cv.putText(color_result[y+5:y+45, x+5:x+45], f'{grid[row][col]}', (10,30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv.LINE_AA)

In [44]:
cv.imshow('final', color_result)

In [45]:
result = cv.warpPerspective(color_result, matrix, img.shape[:2][::-1], img, cv.WARP_INVERSE_MAP, borderMode=cv.BORDER_TRANSPARENT)

In [49]:
cv.imshow('final_image', img)